### Tool

A tool is just a python function (API) that is packaged in a way that the LLM can understand and call when needed.

### Agent

An AI agent is an LLM-powered system that can autonomously think, decide and take actions using external tools or APIs to achieve a goal.